In [80]:
# %%
query="""The address generated by the processor is referred to as ___?"""


In [81]:
# %%
%pip install -qU langchain-text-splitters
%pip install pypdf
%pip install sentence_transformers
%pip install groq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [82]:
from groq import Groq

In [83]:
client = Groq(api_key="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")

In [84]:
# %%
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    separators=["\n\n", "\n", " ", ""],
)
from pypdf import PdfReader

reader = PdfReader("pdf1.pdf")
pages=len(reader.pages)
document_data=""
for i in range(pages):
    document_data+=" "+reader.pages[i].extract_text()
texts = text_splitter.create_documents([document_data])
texts2=[]
for i in texts:
    texts2.append(i.page_content)



In [85]:
# %%
print(texts2)


['COMPUTER ORGANIZATIO N  \n1 \n \nUNIT -II: THE MEMORY  SYSTEM', '1.Basic  Concepts:', 'The maximum  size of the  memory  that can  be used  in any computer is  determined  by the', 'addressing  scheme.', 'If MAR is k bits long and MDR is n bits long, then the memory may contain upto 2K', 'addressable locations and the n -bits of data are transferred between the memory and  \nprocessor.', 'processor.  \nThis transfer  takes  place over the processor  bus. \nThe processor  bus has,', '• Address  Line \n• Data Line \n• Control  Line (R/W,  MFC – Memory  Function  Completed)', 'The control line  is used for  co-ordinating  data transfer.', 'The processor  reads  the data from the memory  by loading  the address of  the required', 'memory location into  MAR and setting  the R/W line to 1.', 'The memory  responds  by placing  the data from the addressed  location  onto the data lines', 'and confirms  this action by  asserting MFC  signal.', 'Upon receipt  of MFC signal,  the processor  loa

In [86]:
# %%
%pip install langchain_community


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [87]:
# %%
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
doc_result = embeddings.embed_documents(texts2)


In [88]:
# %%
vectors = []

for i in range(len(doc_result)):
    vectors.append(
        {
            "id": f"vec{i+1}", 
            "values": doc_result[i], 
            "metadata": {'text': texts2[i]}
        }
    )



In [89]:
# %%
%pip install pinecone_client


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [92]:
# %%
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="404965f2-a54e-426b-964f-2bff9194bdbd")
pc.create_index(
    name="quickstart",
    dimension=384, 
    metric="euclidean",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)


In [93]:
# %%
index = pc.Index("quickstart")
index.upsert(
    vectors=vectors,
    namespace= "ns1"

)


{'upserted_count': 45}

In [94]:
# %%
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device=device)
model


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [95]:
# %%
xq = model.encode(query).tolist()
xc=index.query(
    namespace="ns1",
    vector=xq,
    top_k=10,
    include_values=True,
    include_metadata=True
)
relevant_chunk=""
for result in xc['matches']:
    relevant_chunk+=" "+result['metadata']['text']
print(relevant_chunk)


 The address  generated by  the processor  is referred  to as a virtual  / logical address. 2 
 
The processor writes the data into the memory location by loading the address of this The address generated by the processor does not directly specify the physical locations in The processor  reads  the data from the memory  by loading  the address of  the required processor.  
It holds  the currently  active  segments of a  program  and their data. addressable locations and the n -bits of data are transferred between the memory and  
processor. Thus,  during  every  memory  cycle,  an address  processing  mechanism  determines  whether processor.  
This transfer  takes  place over the processor  bus. 
The processor  bus has, As the active portion of the virtual address space changes during program execution, the It is a small,  fast memory  that is inserted  between  the larger  slower main memory  and the


In [96]:
# %%
completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {
            "role": "system",
            "content": f"answer EXACTLY based on [{relevant_chunk}] - try to use the same words"
        },
        {
            "role": "user",
            "content": query
        }
    ],
    temperature=0,
    max_tokens=8192,
    top_p=1,
    stream=True,
    stop=None,
)
relevant_response=""
for chunk in completion:
    relevant_response+=chunk.choices[0].delta.content or ""
print(relevant_response)


The address generated by the processor is referred to as a virtual/logical address.


In [97]:
# %%
pc.delete_index('quickstart')